In [1]:
!pip install pandas scikit-learn

In [2]:
!pip install opencv-python pandas mediapipe

In [3]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import csv

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.3, model_complexity=2
)

In [5]:
#functions to write and append rows to our csv file
def write_row(row, mode='w'):
    with open(
        "C:/Users/cosmo/Projects/YogaPoseHelper/app/data/yogaposes.csv",
        mode=mode,
        newline="",
    ) as f:
        writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(row)

def append_row(row):
    write_row(mode='a')


def extract_coords_as_row(frame,pose):
    """
    returns pose coordinates in the form : [x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,y3....]
    where x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,y3 are the coordinates of landmarks 0,1,2,3...
    """

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = pose.process(img)

    if results.pose_landmarks:
        try:
            landmarks = results.pose_landmarks.landmark
            row = list(
                np.array(
                    [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
                ).flatten()
            )

            return row

        except Exception as e:
            print(e)
            print("Error while exporting coords")
    
    return None

In [6]:
df = pd.read_csv('C:\\Projects\\YogaPoseHelper\\app\\data\\yogaposes.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Projects\\YogaPoseHelper\\app\\data\\yogaposes.csv'

In [ ]:
X = df.drop('class',axis=1)
y = df['class']

X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
clf = RandomForestClassifier()

model = clf.fit(X_train,y_train)

testing our model:

In [ ]:
row = extract_coords_as_row(cv2.imread('C:\\Users\\cosmo\\Projects\\YogaPoseHelper\\app\\data\\kaggle\\DATASET\\TEST\\plank\\00000001.jpg'),pose)
print(clf.predict(pd.DataFrame(columns=df.columns[1:],data=[row])))
print(clf.score(X_test,y_test))

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# Live Preview:

(also full body should be visible in webcam for accurate classification)

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, frame = cap.read()
    pose_name = "No pose detected"

    if success:
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(img)

        if results.pose_landmarks:
            # predict the pose name
            row = extract_coords_as_row(frame, pose)
            pose_name = clf.predict(pd.DataFrame(columns=df.columns[1:],data=[row]))[0]

            # draw the landmarks
            mp_drawing.draw_landmarks(
                image=img,
                landmark_list=results.pose_landmarks,
                connections=mp_pose.POSE_CONNECTIONS,
            )

            # make the final image equal to the image we drew on
            frame = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    cv2.putText(frame, pose_name, (5, 20), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
    cv2.imshow("webcam feed", frame)

    if cv2.waitKey(10) & 0xFF == ord("q"):
        break


cap.release()
cv2.destroyAllWindows()